# Sorting-Answer 

#### By Dae Hyeun (Isaac) Cheong 

This document has a code to sort a free response question into 4 categories: 
* **MC**: Most likely to be a correct answer
* **CC**: Likely to be a correct answer but need more careful look 
* **MI**: Most likely to be a incorrect answer but need more careful look
* **I**: Incorrect answer that need 'no check' (Most likely to be a blank answer) 

In [2]:
{
    "tags": [
        "hide-cell"
    ]
}


library(knitr)
library(dplyr)
library(readxl)
Items_28_and_30_2 <- read_excel("Items 28 and 30-2.xls")
translated_pc <- read_excel("translated_pc.xlsx")

ERROR: Error in parse(text = x, srcfile = src): <text>:2:13: unexpected '['
1: {
2:     "tags": [
               ^


#### Methodology

To explain the methodology on how to sort an answer, I will use the question about water bubbles, which coresspond to question 4 of the science assessment. To the original dataset, the answer to the question 4 correspond to the column name "pc"

![AnswerRuburic](scoringrubric.png)